# Sending a pulse through a 3 cavity-2 qubit system, <mark>without thermal losses</mark> 

1. **Introduction**
2. **Problem parameters**
3. **Setting up operators and Hamiltonian's**
5. **Evolving the system with time**
4. **Results**

<u>Author</u> : Soumya Shreeram (shreeramsoumya@gmail.com)<br>
<u>Supervisor</u> : Yu-Chin Chao (ychao@fnal.gov) <br>
<u>Date</u>$\ \ \ \$: 8th July 2019<br>

This script was coded as part of the Helen Edwards Summer Internship program at Fermilab.

## 1. Introduction

A multi-mode QED architecture is explored as described in by [McKay et *al*](http://schusterlab.uchicago.edu/static/pdfs/McKay2015.pdf). The hamiltonian for such a system with two qubits with frequencies $v_{Q,1}$, $v_{Q,2}$, and $n$ mode filter can be described as the sum of the qubit Hamiltonian, $\hat{H}_Q$, the filter Hamiltonian, $\hat{H}_F$, and the qubit-filter coupling Hamiltonian, $\hat{H}_{Q-F},$
$$ \hat{H} = \hat{H_Q} + \hat{H_F} + \hat{H}_{Q-F} $$

$$ \hat{H_Q} = h\ v_{Q,1}\ \frac{\hat{ \sigma}^z_1}{2} + h\ v_{Q,2}\ \frac{\hat{ \sigma}^z_2}{2}$$

$$ \hat{H}_{F} = \sum_{i=1}^{n}h\ v_{F}\ \hat{a}^{\dagger}_i \hat{a}_i +  \sum_{i=2}^{n}h\ g_{F}\ (\hat{a}^{\dagger}_i \hat{a}_{i-1} + \hat{a}^{\dagger}_{i-1} \hat{a}_i)$$

$$ \hat{H}_{Q-F} = h\ g_{Q1,F}\ (\hat{a}^{\dagger}_1 \hat{\sigma}^-_1 + \hat{a}_1 \hat{\sigma}^+_1) + h\ g_{Q2,F}\ (\hat{a}^{\dagger}_n \hat{\sigma}^-_2 + \hat{a}_n \hat{\sigma}^+_2)$$

where $\hat{\sigma}^{+(-)}$ is the raising and lowering operator for the qubit, $\hat{a}_i$ creates a photon in the $i^{th}$ resonantor, $g_F$ is the filter-filter coupling, and $g_{Q,F}$ is the qubit-filter coupling.

Here we must also account for the interaction of the quantum state with it's environment. This can be represented by a non-Hermitian term in the Hamiltonian such that,
$$\displaystyle H_{\rm eff}(t) = H(t) - \frac{i\hbar}{2}\sum_n c_n^\dagger c_n$$
where $c_n$ is the collapse operator

The code calculates the eigen modes for such a system when the qubit 1 frequency is changed. 

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from math import pi
from qutip import *

## 2. Problem parameters
Here we use $\hbar=1$; the coupling terms are redefined with a multiple of $2\pi$ before them for convinience.

In [ ]:
w_q1 = 2*pi*6.85;    # Qubit 1 frequency
w_q2 = 2*pi*6.82;    # Qubit 2 frequency: generally in the range from 1-9 GHz
w_f = 2*pi*7.1      # Resonator/ Filter frequency
g_f1 = 2*pi*11.8    # Filter-filter coupling
g_f2 = 2*pi*19.8 
g_q1f = 2*pi*13.5   # qubit-fitler coupling
g_q2f = 2*pi*14.4
numF = 3             # number of filters
N = 2                # number of fock states 

# resonant SQRT iSWAP gate
T0_1 = 20
T_gate_1 = (1*pi)/(4 * g1)

# resonant iSWAP gate
T0_2 = 60
T_gate_2 = (2*pi)/(4 * g2)

## 3. Setting up the operators and the Hamiltonian's

For every qubit: <br> <br>
**sm** $\ \rightarrow \ \hat{\sigma}^{+(-)}$ is the raising and lowering operator of the *qubit* <br>
**sz** $\ \ \rightarrow \ \sigma_z $ is the Pauli-z matrix of the *qubit* <br>
**n** $\ \ \ \rightarrow \ n$ is the number operator

In [1]:
def Hqubit(sz, w_q):
  """ 
  Comuptes the qubit hamiltonian
  @param sz :: Pauli's z matrix
  @param w_q :: qubit frequency
  """
  return 0.5*sz*w_q

def numOp(m):
  """
  Computes the number operator 
  @param loweringMat :: lowering matrix operator for a system
  """
  return m.dag()*m

def sumDagger(m):
  """
  Computes sum of operator m and m.dag()
  @param a1 :: destruction operator 1
  @param a2 :: destruction operator 2
  """
  return m+m.dag()

### 3.1 Operators & Hamiltonian's for the cavities and qubit's

In [ ]:
# cavity 1, 2, 3 destruction operators
a1 = tensor(destroy(N), qeye(N), qeye(N), qeye(2), qeye(2))
a2 = tensor(qeye(N), destroy(N), qeye(N), qeye(2), qeye(2))
a3 = tensor(qeye(N), destroy(N), qeye(N), qeye(2), qeye(2))

# operators for qubit 1 
sm1 = tensor(qeye(N), qeye(N), qeye(N), destroy(2), qeye(2))
sz1 = tensor(qeye(N), qeye(N), qeye(N), sigmaz(), qeye(2))    
n1 = sm1.dag() * sm1

# operators for qubit 2
sm2 = tensor(qeye(N), qeye(N), qeye(N), qeye(2), destroy(2))
sz2 = tensor(qeye(N), qeye(N), qeye(N), qeye(2), sigmaz())
n2 = sm2.dag() * sm2

In [ ]:
# Qubit Hamiltonians (Hq1+Hq2)
Hq1 = Hqubit(sz1, w_q1)       
Hq2 = Hqubit(sz2, w_q2)

# Filter Hamiltonians (refer formula in the Introduction)
H_f1 = tensor(destroy(N),create(N),qeye(N),qeye(2), qeye(2)) + tensor(create(N),destroy(N),qeye(N),qeye(2), qeye(2))
H_f2 = tensor(qeye(N),create(N),destroy(N),qeye(2), qeye(2)) + tensor(qeye(N),destroy(N),create(N),qeye(2), qeye(2))

Hf = w_f1*numOp(a1) + w_f2*numOp(a1) + w_f3*numOp(a1)+ g_f1*H_f1+g_f2*H_f2   

# Qubit-Filter Hamiltonian
H_qf1 = tensor(destroy(N),qeye(N), qeye(N), sigmap(),qeye(2))
H_qf2 = tensor(qeye(N),qeye(N), destroy(N), qeye(2), sigmap())

Hqf = g_q1f*sumDagger(H_qf1) + g_q2f*sumDagger(H_qf2)
 
H = Hq1 + Hq2 + Hf + Hqf   # Resultant Hamiltonian
H_q1 = Hq2 + Hf + Hqf      # Qubit 1 independent Hamiltonian

### 3.2 Generating the pulse wave

In [ ]:
def step_t(w1, w2, t0, width, t):
    """
    Step function that goes from w1 to w2 at time t0
    as a function of t. 
    """
    return w1 + (w2 - w1) * (t > t0)
  
def wf_t(t, args=None):
  #No change in the frequency with time 
    return w_f
  
def w1_t(t, args=None):
  # Pulse generated at qubit 1 freq. at T0_1
    return w_q1 + step_t(0.0, w_f-w_q1, T0_1, width, t) - step_t(0.0, wc-w1, T0_1+T_gate_1, width, t)

def w2_t(t, args=None):
    return w_q2 + step_t(0.0, w_f-w_q2, T0_2, width, t) - step_t(0.0, wc-w2, T0_2+T_gate_2, width, t) 
  
